# 1. Introduction

In [ ]:
import pandas as pd

# 2. Basic data processing

# 3. Data exploration

# 4. Traditional modelling

# 5. Global modelling

# 6. Deep learning

# 7. Approach comparison

# 8. Conclusions